# Notebook for running inference on Google Colab

* Setup
* Create pipeline
* Run pipeline for each target var

### Setup

In [12]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/CVSS_Prediction/cvss_predictor.py .
!cp /content/drive/MyDrive/CVSS_Prediction/CustomEvalTrainer.py .
!pip install transformers datasets torch cvss
!nvcc --version
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Mon May 16 13:20:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    

In [16]:
from typing import Dict
import torch
import pandas as pd
from transformers import TextClassificationPipeline, AutoTokenizer, DistilBertTokenizerFast, AutoModelForSequenceClassification
from transformers.pipelines.base import GenericTensor
from tqdm.auto import tqdm
from cvss import CVSS3
from sklearn.metrics import mean_squared_error, mean_absolute_error

label_replacements = {
    'av': {
        'c2i': {'N': 0, 'L': 1, 'A': 2, 'P': 3},
        'i2c': {0: 'N', 1: 'L', 2: 'A', 3: 'P'}
    },
    'ac': {
        'c2i': {'L': 0, 'H': 1},
        'i2c': {0: 'L', 1: 'H'}
    },
    'pr': {
        'c2i': {'L': 0, 'H': 1, 'N': 2},
        'i2c': {0: 'L', 1: 'H', 2: 'N'}
    },
    'ui': {
        'c2i': {'R': 0, 'N': 1},
        'i2c': {0: 'R', 1: 'N'}
    },
    's': {
        'c2i': {'C': 0, 'U': 1},
        'i2c': {0: 'C', 1: 'U'}
    },
    'c': {
        'c2i': {'H': 0, 'N': 1, 'L': 2},
        'i2c': {0: 'H', 1: 'N', 2: 'L'}
    },
    'i': {
        'c2i': {'H': 0, 'N': 1, 'L': 2},
        'i2c': {0: 'H', 1: 'N', 2: 'L'}
    },
    'a': {
        'c2i': {'H': 0, 'N': 1, 'L': 2},
        'i2c': {0: 'H', 1: 'N', 2: 'L'}
    }
}

In [ ]:
if not torch.cuda.is_available():
  print('No GPU detected')
else:
  n = torch.cuda.device_count()
  print(f'{n} GPU detected')
  gpu_info = str(torch.cuda.get_device_properties(0)).replace("_CudaDeviceProperties", "")
  print(f'Device {0}: "{gpu_info}"')

1 GPU detected
Device 0: "(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)"


In [ ]:
dataset_path = "/content/drive/MyDrive/CVSS_Prediction/dataset/distilbert"
base_model = "distilbert-base-uncased"
checkpoint_path = "/content/drive/MyDrive/CVSS_Prediction/checkpoints_distilbert/"
target_vars = ['av', 'ac', 'pr', 'ui', 's', 'c', 'i', 'a']

### Create Pipeline

In [ ]:
class MyPipeline(TextClassificationPipeline):
    def preprocess(self, inputs, **tokenizer_kwargs) -> Dict[str, GenericTensor]:
        return_tensors = self.framework
        return self.tokenizer(inputs, return_tensors=return_tensors, truncation=True, **tokenizer_kwargs)

In [ ]:
df = pd.read_json(f"{dataset_path}/test.json", orient='records', lines=True)
df = df.drop(columns=['input_ids', 'attention_mask'])

In [ ]:
tokenizer: DistilBertTokenizerFast = AutoTokenizer.from_pretrained(base_model)

### Run for each target var

In [ ]:
for target_var in target_vars:
  print('-'*10 + target_var + '-'*10)
  number_of_labels = len(df[target_var].unique())
  df[f'predicted_{target_var}'] = pd.NaT * len(df)
  
  model_path = f"{checkpoint_path}distil_{target_var}/checkpoint-final"
  model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=number_of_labels)
  pipe = MyPipeline(model=model, tokenizer=tokenizer, function_to_apply="softmax", device=0)

  replacements = label_replacements[target_var]['i2c']

  incorrect = 0

  total_n = len(df)

  for index, row in tqdm(df[:total_n].iterrows(), total=total_n):
    #print('-'*10 + f"{x}/{total_n}" + '-'*10)
    #print(row['id'] + ':' + target_var + '=' + row[target_var])
    out = pipe(row['text'])
    prediction = out[0]['label']
    prediction = prediction.replace('LABEL_', '')
    prediction = replacements[int(prediction)]
    df.at[index, f'predicted_{target_var}'] = prediction
    if prediction is not row[target_var]: incorrect += 1
    #print(f'Prediction: {prediction}')

  print(f"Incorrect: {incorrect}")

----------av----------


  0%|          | 0/25434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Incorrect: 1762
----------ac----------


  0%|          | 0/25434 [00:00<?, ?it/s]

...

Incorrect: 2866
----------i----------


  0%|          | 0/25434 [00:00<?, ?it/s]

Incorrect: 2633
----------a----------


  0%|          | 0/25434 [00:00<?, ?it/s]

Incorrect: 2429


### Inspect results

In [ ]:
df.head(50)

,id,text,av,ac,pr,ui,s,c,i,a,score,predicted_av,predicted_ac,predicted_pr,predicted_ui,predicted_s,predicted_c,predicted_i,predicted_a
0,CVE-2016-0003,Microsoft Edge allows remote attackers to exec...,N,L,N,R,C,H,H,H,9.6,N,H,N,R,U,H,H,H
1,CVE-2016-0003,This vulnerability allows remote attackers to ...,N,L,N,R,C,H,H,H,9.6,N,L,N,R,U,H,H,H
2,CVE-2016-0011,Microsoft SharePoint Server 2013 SP1 and Share...,N,L,L,R,C,L,L,N,5.4,N,L,L,R,C,L,L,N
3,CVE-2016-0016,"Microsoft Windows Vista SP2, Windows Server 20...",L,L,L,N,U,H,H,H,7.8,L,L,L,N,U,H,H,H
4,CVE-2016-0021,"Microsoft InfoPath 2007 SP3, 2010 SP2, and 201...",L,L,N,R,U,H,H,H,7.8,L,L,N,R,U,H,H,H
5,CVE-2016-0025,"Microsoft Word 2007 SP3, Office 2010 SP2, Word...",L,L,L,R,U,H,H,H,7.3,L,L,N,R,U,H,H,H
6,CVE-2016-0030,Cross-site scripting (XSS) vulnerability in Ou...,N,L,N,R,C,L,L,N,6.1,N,L,N,R,C,L,L,N
7,CVE-2016-0034,Microsoft Silverlight 5 before 5.1.41212.0 mis...,N,L,N,R,U,H,H,H,8.8,N,L,N,R,U,H,H,H
8,CVE-2016-0037,The forms-based authentication implementation ...,N,L,N,N,U,N,N,H,7.5,N,L,N,N,U,N,N,H
9,CVE-2016-0041,"Microsoft Windows Vista SP2, Windows Server 20...",L,L,L,N,U,H,H,H,7.8,L,L,L,N,U,H,H,H


In [23]:
total_n = len(df)

for index, row in tqdm(df[:total_n].iterrows(), total=total_n):
  vector = f"CVSS:3.0/AV:{row['predicted_av']}/AC:{row['predicted_ac']}/PR:{row['predicted_pr']}/UI:{row['predicted_ui']}/S:{row['predicted_s']}/C:{row['predicted_c']}/I:{row['predicted_i']}/A:{row['predicted_a']}"
  c = CVSS3(vector)
  base_score = c.scores()[0]
  df.at[index, 'predicted_score'] = base_score

  0%|          | 0/25434 [00:00<?, ?it/s]

In [24]:
df.head()

,id,text,av,ac,pr,ui,s,c,i,a,score,predicted_av,predicted_ac,predicted_pr,predicted_ui,predicted_s,predicted_c,predicted_i,predicted_a,predicted_score
0,CVE-2016-0003,Microsoft Edge allows remote attackers to exec...,N,L,N,R,C,H,H,H,9.6,N,H,N,R,U,H,H,H,7.5
1,CVE-2016-0003,This vulnerability allows remote attackers to ...,N,L,N,R,C,H,H,H,9.6,N,L,N,R,U,H,H,H,8.8
2,CVE-2016-0011,Microsoft SharePoint Server 2013 SP1 and Share...,N,L,L,R,C,L,L,N,5.4,N,L,L,R,C,L,L,N,5.4
3,CVE-2016-0016,"Microsoft Windows Vista SP2, Windows Server 20...",L,L,L,N,U,H,H,H,7.8,L,L,L,N,U,H,H,H,7.8
4,CVE-2016-0021,"Microsoft InfoPath 2007 SP3, 2010 SP2, and 201...",L,L,N,R,U,H,H,H,7.8,L,L,N,R,U,H,H,H,7.8


In [25]:
df.to_csv("/content/drive/MyDrive/CVSS_Prediction/predictions.csv")

In [ ]:
score_true = df['score']
score_predicted = df['predicted_score']
print(len(score_true))
print(len(score_predicted))

In [ ]:
mean_squared_error(y_true=score_true, y_pred=score_predicted)

In [ ]:
mean_absolute_error(y_true=score_true, y_pred=score_predicted)